In [2]:
!pip install pytorch-pretrained-bert

In [3]:
!mkdir classification_binary
!mkdir classification_fine_grained

mkdir: cannot create directory ‘classification_binary’: File exists
mkdir: cannot create directory ‘classification_fine_grained’: File exists


Upload datasets

## Classification

In [0]:
# TASK = "ASP_BINARY"
TASK = "ASP_MULTI"

### Loading data

In [0]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# names = ["object_a", "object_b", "most_frequent_label", "sentence"]
# accepted_rows = [2,3,9, 17]

# df = pd.read_csv("ACQuA - CompArg - release v1 - dataset of comparative sentences - all-data.csv", usecols=names)
# df_X = pd.DataFrame(df, columns=['object_a', "object_b", "sentence"])
# df_y = pd.DataFrame(df, columns=['most_frequent_label'])

# df_train, df_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)
# df_train['most_frequent_label'] = y_train['most_frequent_label']
# df_test['most_frequent_label'] = y_test['most_frequent_label']

# df_train.shape

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
names = ["object_a", "object_b", "most_frequent_label", "sentence"]
accepted_rows = [2,3,9, 17]

df_train = pd.read_csv("train_data.csv", usecols=names)
df_test = pd.read_csv("test_data.csv", usecols=names)



In [6]:
df_train

,object_a,object_b,sentence,most_frequent_label
0,Dell,Intel,"Frankly, like a procession of formerly-consist...",NONE
1,mit,harvard,"mit, harvard",NONE
2,Microsoft,Sony,It also didn't hurt my faith in Microsoft when...,BETTER
3,Ethernet,Bluetooth,You can probably find the Onkyo TX-NR717 for m...,NONE
4,Toyota,Ford,I liked the Toyota over the Ford and it looks ...,BETTER
...,...,...,...,...
5754,Chevrolet,Ford,I drive a Chevrolet Trailblazer...not much dif...,BETTER
5755,PostgreSQL,Oracle,migration tool and an editor of your choice Ex...,NONE
5756,Java,Python,Regarding this C/C++/D/Java/C#/etc are worse a...,WORSE
5757,JavaScript,PHP,"Solid skills in shader writing, python, C/C++,...",NONE


In [8]:
df_test.shape

(1440, 4)

In [9]:
df_train.head()

,object_a,object_b,sentence,most_frequent_label
0,Dell,Intel,"Frankly, like a procession of formerly-consist...",NONE
1,mit,harvard,"mit, harvard",NONE
2,Microsoft,Sony,It also didn't hurt my faith in Microsoft when...,BETTER
3,Ethernet,Bluetooth,You can probably find the Onkyo TX-NR717 for m...,NONE
4,Toyota,Ford,I liked the Toyota over the Ford and it looks ...,BETTER


### Data preparing

In [0]:
def target_to_int(value):
  if value == 'BETTER':
    return 0
  elif value == 'WORSE':
    return 1
  elif value == 'NONE':
    return 2

def get_bert_format(df):
  df_bert = pd.DataFrame({
      'id':range(len(df)),
      'label': df['most_frequent_label'].apply(target_to_int),
      'alpha':['a']*df.shape[0],
      'text': df["object_a"] + ' [SEP] ' + df["object_b"] + ' [SEP] ' + df["sentence"]
      })
  return df_bert


df_train_bert = get_bert_format(df_train)
df_test_bert = get_bert_format(df_test)

In [60]:
df_train_bert.head(15)

,id,label,alpha,text
0,0,2,a,"Dell [SEP] Intel [SEP] Frankly, like a process..."
1,1,2,a,"mit [SEP] harvard [SEP] mit, harvard"
2,2,0,a,Microsoft [SEP] Sony [SEP] It also didn't hurt...
3,3,2,a,Ethernet [SEP] Bluetooth [SEP] You can probabl...
4,4,0,a,Toyota [SEP] Ford [SEP] I liked the Toyota ove...
5,5,0,a,Perl [SEP] Java [SEP] - Perl has constructs th...
6,6,2,a,nixon [SEP] reagan [SEP] Obama is much worse t...
7,7,2,a,Advil [SEP] Motrin [SEP] OTC pain killers (Adv...
8,8,0,a,Windows 7 [SEP] Windows 8 [SEP] On Balance Mic...
9,9,2,a,Python [SEP] Ruby [SEP] I'm just wondering if ...


In [8]:
DATA_DIR = "comparative/"

!mkdir comparative

df_train_bert.to_csv(DATA_DIR + 'train.tsv', sep='\t', index=False, header=False)
df_test_bert.to_csv(DATA_DIR + 'test.tsv', sep='\t', index=False, header=False)

mkdir: cannot create directory ‘comparative’: File exists


### BERT

In [0]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import convert_examples_to_features


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

TASK_NAME = 'comparative'

OUTPUT_DIR = f'outputs/{TASK_NAME}/'
CACHE_DIR = 'cache/'

MAX_SEQ_LENGTH = 30

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 20
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
output_mode = OUTPUT_MODE

cache_dir = CACHE_DIR

In [13]:
if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

ValueError: ignored

In [0]:
processor = MultiClassificationProcessor()

train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

In [0]:
label_list = processor.get_labels()
num_labels = len(label_list)

In [0]:
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

In [19]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 5759 examples..
Spawning 1 processes..


In [0]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [43]:
# Load pre-trained model (weights)
# model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)
model = BertForSequenceClassification.from_pretrained("outputs/comparative/pytorch_model.bin", cache_dir='outputs/comparative/', num_labels=len(label_list))

ReadError: ignored

In [22]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [0]:
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)

In [0]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [0]:
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

In [0]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE, drop_last=True)

In [28]:
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)

        if OUTPUT_MODE == "classification":
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0.444851

Epoch:   3%|▎         | 1/30 [00:36<17:51, 36.93s/it]

0.350098


0.562036

...

Epoch:  93%|█████████▎| 28/30 [17:23<01:14, 37.37s/it]

0.000437

Epoch:  97%|█████████▋| 29/30 [18:00<00:37, 37.38s/it]

0.000373

Epoch: 100%|██████████| 30/30 [18:37<00:00, 37.41s/it]

In [29]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

'outputs/comparative/vocab.txt'

In [0]:
!mv outputs/comparative/config.json bert_config.json
!mv outputs/comparative/pytorch_model.bin pytorch_model.bin

In [0]:
!tar -czvf cache/comp.tar.gz bert_config.json pytorch_model.bin

In [0]:
!mv bert_config.json outputs/comparative/config.json
!mv pytorch_model.bin outputs/comparative/pytorch_model.bin

### Evaluation

In [0]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import convert_examples_to_features


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
names = ["object_a", "object_b", "most_frequent_label", "sentence"]
accepted_rows = [2,3,9, 17]

In [0]:
TASK_NAME = 'comparative'
OUTPUT_DIR = f'outputs/{TASK_NAME}/'
CACHE_DIR = 'cache/'

MAX_SEQ_LENGTH = 30

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 20
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'
output_mode = OUTPUT_MODE
DATA_DIR = "comparative/"
BERT_MODEL = "comp.tar.gz"
CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
df_test = pd.read_csv("test_data.csv", usecols=names)


In [0]:
def target_to_int(value):
  if value == 'BETTER':
    return 0
  elif value == 'WORSE':
    return 1
  elif value == 'NONE':
    return 2

def get_bert_format(df):
  df_bert = pd.DataFrame({
      'id':range(len(df)),
      'label': df['most_frequent_label'].apply(target_to_int),
      'alpha':['a']*df.shape[0],
      'text': df["object_a"] + ' [SEP] ' + df["object_b"] + ' [SEP] ' + df["sentence"]
      })
  return df_bert


df_test_bert = get_bert_format(df_test)
df_test_bert.to_csv(DATA_DIR + 'test.tsv', sep='\t', index=False, header=False)

In [0]:
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)


In [0]:
processor = MultiClassificationProcessor()

In [0]:
eval_examples = processor.get_test_examples(DATA_DIR)
eval_examples_len = len(eval_examples)
label_list = processor.get_labels()
num_labels = len(label_list)

In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

In [10]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

Preparing to convert 1440 examples..
Spawning 1 processes..


In [0]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

In [0]:
if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

In [0]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(CACHE_DIR + BERT_MODEL, cache_dir=CACHE_DIR, num_labels=len(label_list))

In [16]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [17]:
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    # create eval loss and other metric required by the task
    if OUTPUT_MODE == "classification":
        loss_fct = CrossEntropyLoss()
        tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))

    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
if OUTPUT_MODE == "classification":
    preds = np.argmax(preds, axis=1)

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def report_scores(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    pr = precision_score(y_true, y_pred, average='weighted')
    re = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy: {:.2f}".format(acc * 100))
    print("Precision: {:.2f}".format(pr * 100))
    print("Recall: {:.2f}".format(re * 100))
    print("F1: {:.2f}".format(f1 * 100))
    target_names = ['BETTER', 'WORSE', 'NONE']
    print(classification_report(y_true, y_pred, target_names=target_names))

In [19]:
report_scores(all_label_ids.numpy(), preds)

Accuracy: 84.24
Precision: 84.02
Recall: 84.24
F1: 84.12
              precision    recall  f1-score   support

      BETTER       0.72      0.72      0.72       273
       WORSE       0.58      0.53      0.55       119
        NONE       0.90      0.91      0.91      1048

    accuracy                           0.84      1440
   macro avg       0.73      0.72      0.73      1440
weighted avg       0.84      0.84      0.84      1440



In [0]:
report_scores(all_label_ids.numpy(), preds)

Accuracy: 84.86
Precision: 84.73
Recall: 84.86
F1: 84.79
              precision    recall  f1-score   support

      BETTER       0.74      0.71      0.73       273
       WORSE       0.58      0.57      0.57       119
        NONE       0.91      0.92      0.91      1048

    accuracy                           0.85      1440
   macro avg       0.74      0.73      0.74      1440
weighted avg       0.85      0.85      0.85      1440

